Nihar Lohar
Btech AI
LAB 5


# Aim:
 Exploring basic concepts of LangChain

Task 1: Creating API Key, setting up an environment and using LangChain’s model primitive

· Create an account and get your API key (Mistral AI, Gemini or any other)

· Set up the environment (use either Colab or Jupyter notebook)

· Install LangChain (Installation steps will vary based on the environment)

· Maka first call to the specific llm model using LangChain’s model primitive (e.g. ChatMistralAI from langchain_mistralai). Write a prompt and get the response.

Task 2: Using LangChain PromptTemplates

· Craft a prompt and test it.

· Build a chain (Prompt → Model)

· Add an output parser (text only)

· Rerun with temperature=0.1 vs 0.9. How do tone and variety change?

Task 3: Structured JSON (JSON output parsing)

· from langchain_core.output_parsers import JsonOutputParser

· Write a suitable prompt

· Use parser = JsonOutputParser()

· Create a json_chain

· Show the output

· Explore how you can make the model output a fixed schema for your course content (e.g., learning_objectives, prerequisites, time_required_min)

In [1]:
!pip -q install -U langchain langchain-core langchain-community langchain-groq groq pydantic

import os, json
from typing import Dict, Any

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [20]:
import os, getpass
os.environ["GROQ_API_KEY"] = getpass.getpass("Paste your API_KEY : ")


Paste your API_KEY : ··········


In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="qwen/qwen3-32b",
    temperature=0.7
)


In [6]:
resp = llm.invoke("Give me a motivational study tip in one sentence.")
print(resp.content)


<think>
Okay, the user wants a motivational study tip in one sentence. Let me think about what makes a study tip both motivational and effective. They probably need something that's easy to remember and can boost their morale.

First, the tip should encourage consistent effort. Words like "small steps" or "progress" might work. Maybe emphasize that every bit of work adds up. Also, positivity is key here. Phrases like "celebrate every small victory" could help in maintaining motivation. 

I should avoid clichés that might not feel genuine. Instead, focus on the process rather than the outcome. Maybe mention the importance of showing up each day. 

Let me check if there's a common study tip that's both motivational and concise. Oh, the idea of progress over perfection. Yeah, that's a good angle. People often feel overwhelmed by big goals, so focusing on small steps can make it manageable. 

Putting it all together: Something like, "Celebrate every small step forward, because consistent p

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI tutor."),
    ("human", "Summarize {topic} in 3 bullet points.")
])

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"topic":"LangChain basics"}))


<think>
Okay, I need to summarize the basics of LangChain into three bullet points. Let me start by recalling what I know about LangChain.

First, I remember that LangChain is a framework for working with large language models. It probably helps in building applications using LLMs. So one point might be about its purpose as a framework for developing LLM-powered apps.

Next, the second point. LangChain has components like prompts, models, memory, and indexes. Oh right, and it connects them together. Maybe it's about the key components and how they integrate to handle data and memory, making it easier to manage complex workflows.

Third, I think LangChain provides tools for interacting with external data sources and APIs. Like, it can fetch data from a database or use a web search API. So the third bullet could be about the tools for integrating with external systems and data sources to enhance the application's capabilities.

Wait, let me check if I missed anything. Oh, maybe the abili

In [8]:
llm_low = ChatGroq(model="qwen/qwen3-32b", temperature=0.1)
llm_high = ChatGroq(model="qwen/qwen3-32b", temperature=0.9)

chain_low  = prompt | llm_low  | StrOutputParser()
chain_high = prompt | llm_high | StrOutputParser()

q = {"style":"helpful", "topic":"staying focused while studying"}

print("T=0.1 ➜", chain_low.invoke(q))
print("T=0.9 ➜", chain_high.invoke(q))


T=0.1 ➜ <think>
Okay, the user wants a summary of staying focused while studying in three bullet points. Let me start by recalling the key strategies for maintaining focus. First, eliminating distractions is crucial. That includes things like turning off notifications, finding a quiet space, and maybe using tools like website blockers. That's a solid first point.

Next, breaking tasks into smaller chunks. When studying, large tasks can be overwhelming, so using techniques like the Pomodoro method—working in short intervals with breaks—can help maintain concentration. That makes sense as the second point.

Third, staying motivated. This could involve setting clear goals, rewarding oneself after completing tasks, and reminding oneself of the long-term benefits. Maybe also mentioning staying hydrated and taking short breaks to keep energy levels up. That covers the third point.

Wait, should I mention physical aspects like environment or posture? The user asked for three bullet points, so

#Inference:

At temperature 0.1, the answer stays consistent and straight to the point, giving clear but predictable advice. At temperature 0.9, the answer feels more open and creative, adding extra examples and varied wording, though sometimes less focused.

In [23]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

llm_json = ChatGroq(model="qwen/qwen3-32b", temperature=0)
base_parser = JsonOutputParser()
fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=llm_json)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Output only valid JSON with the required keys."),
    ("human",
     """Topic: {topic}
Keys: learning_objectives (string[]), prerequisites (string[]), time_required_min (integer).""")
])

robust_chain = prompt | llm_json | fixing_parser
print(robust_chain.invoke({"topic": "LangChain"}))


{'learning_objectives': ['Understand the core concepts of LangChain and its role in building AI applications', 'Learn how to integrate LangChain with various language models (LLMs)', 'Master the use of LangChain for creating chains, agents, and memory systems', 'Implement LangChain tools for data retrieval, processing, and response generation', 'Troubleshoot common issues and optimize LangChain workflows'], 'prerequisites': ['Basic knowledge of Python programming', 'Familiarity with API concepts and RESTful services', 'Understanding of machine learning fundamentals', 'Experience with prompt engineering or working with LLMs', 'Installation of required libraries (e.g., langchain, transformers)'], 'time_required_min': 180}


In [24]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class CourseSchema(BaseModel):
    learning_objectives: list[str] = Field(description="Learning objectives of the course")
    prerequisites: list[str] = Field(description="What the student should already know")
    time_required_min: int = Field(description="Estimated time in minutes")

parser = PydanticOutputParser(pydantic_object=CourseSchema)

schema_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a course generator."),
    ("human", "Generate course info for {topic}.\n{format_instructions}")
]).partial(format_instructions=parser.get_format_instructions())

schema_chain = schema_prompt | llm | parser

print(schema_chain.invoke({"topic":"Introduction to LangChain"}))


learning_objectives=['Understand the core concepts and components of LangChain', 'Learn how to install and set up LangChain in a Python environment', 'Explore the integration of LangChain with various language models (LLMs)', 'Develop skills to create and manage prompts for effective model interactions', 'Implement memory and state management in LangChain applications', "Build simple chains and agents using LangChain's modular components", 'Apply LangChain tools for database and API integrations'] prerequisites=['Basic knowledge of Python programming', 'Familiarity with machine learning and large language models (LLMs)', 'Understanding of API concepts and RESTful services', 'Experience with Jupyter Notebooks or Python IDEs', 'Optional: Prior exposure to frameworks like Hugging Face or TensorFlow'] time_required_min=120


#What does LangChain give you beyond calling the HTTP API directly?

LangChain provides a higher-level framework so you don’t have to manage raw API calls yourself. It makes it easier to:

Swap between different model providers with minimal code changes.

Build structured workflows using PromptTemplates, chains, and parsers instead
of string-concatenated prompts.

Maintain conversation history (memory) without manually tracking it.

Enforce reliable outputs as text, JSON, or typed schemas.

Add post-processing, retrieval, or tool use in a better way.


#When would you prefer plain text vs structured (JSON) outputs?
I would prefer plain text when the response is meant to be read directly by people, for example in explanations, summaries, or general study help.

I would prefer structured JSON output when the response needs to be used by another system or application, such as storing course details, creating checklists, or feeding data into a dashboard.